In [1]:
import h5py
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
from skimage.transform import resize, rescale
from glob import glob
import pandas as pd

sns.set_style('darkgrid')

In [2]:
def plot_frame(frame):
    times = [i for i in range(1024)]

    plt.figure(figsize=(15, 10))

    for channel_id in range(128):
        plt.plot(times, frame[:, channel_id].flatten())

    plt.grid(True)
    plt.legend()
    plt.show()

# Loading

In [3]:
def load_exps_and_labels(detections):
    all_exps, all_labels = [], []

    counter = 0

    for exp_name, data in detections.groupby('Exp'):
        counter += 1

        try:
            file = h5py.File(f'../../data/real/{exp_name}.h5', 'r')
        except:
            print(f'Couldnt load exp {exp_name}')
            continue

        exp = []
        
        for channel_id in range(1, 17):
            channel = file[f'Chn{channel_id}']
            exp.append(channel)
        
        exp = np.swapaxes(exp, 1, 0)

        chirps_ids = []

        for _, (frame_id, depth) in data[['Frame', 'Depth']].iterrows():
            chirps_ids = chirps_ids + list(np.linspace(frame_id*127, frame_id*128+127, 128, dtype=np.uint16))
            all_labels = all_labels + list(np.repeat(depth, 128))
            #chirps_ids.append(int(frame_id*127 + 63))
            #all_labels.append(depth)

        exp = exp[chirps_ids]

        all_exps = all_exps + list(exp)

        #if counter > 2:
        #    break
    
    all_exps = np.array(all_exps)
    all_labels = np.array([all_labels]).T
    #all_exps = np.swapaxes(all_exps, 1, 0)
    all_exps[:, :, :50] = 0
    
    return all_exps.astype(np.float32), all_labels.astype(np.float32)
        

In [23]:
FILE_NAME = 'EXP_17_R'

In [5]:
all_detections = pd.read_csv('../../data/real/TABULAR_CSV_ALL_CLASSES.csv')

In [20]:
detections = all_detections[all_detections['Exp'].str.contains('EXP_17_R')].query('Class == "Motorcycle" and Depth <= 25')

In [19]:
pd.unique(detections['Exp'])

array(['EXP_17_C', 'EXP_17_D', 'EXP_17_E', 'EXP_17_F', 'EXP_17_S',
       'EXP_17_T', 'EXP_17_L', 'EXP_17_B', 'EXP_17_I', 'EXP_17_O',
       'EXP_17_G', 'EXP_17_M', 'EXP_17_P', 'EXP_17_R', 'EXP_17_H',
       'EXP_17_Q'], dtype=object)

In [21]:
#raw_data = load_exps([f'../../data/real/{FILE_NAME}.h5'])
raw_data, labels = load_exps_and_labels(detections)

In [22]:
labels.shape

(9984, 1)

In [10]:
np.save(f'../../data/preprocessed/{FILE_NAME}_chirps.npy', raw_data)

In [11]:
raw_data = np.swapaxes(raw_data, 2, 1)

In [12]:
raw_data.shape

(101120, 1024, 16)

# Scale

## Data Scale

In [13]:
print(f'Mean: {raw_data.mean()}, Std: {raw_data.std()}')

Mean: 183.12559509277344, Std: 973.3739013671875


In [14]:
data_min = raw_data.min()
data_max = raw_data.max()

In [15]:
print(f'Min: {data_min}, Max: {data_max}')

Min: -3884.0, Max: 4772.0


In [16]:
a, b = -1, 1

In [17]:
np.subtract(raw_data, data_min, out=raw_data)
np.multiply(raw_data, (b - a), out=raw_data)
np.divide(raw_data, (data_max - data_min), out=raw_data)
raw_data = np.add(raw_data, a, out=raw_data)

In [18]:
normalized_mean = raw_data.mean()

In [19]:
print(f'Mean: {normalized_mean}')

Mean: -0.06028229370713234


## Label Scale

### Label Generator Scale

In [20]:
labels_mean = labels.mean()
labels_std = labels.std()

In [21]:
print(f'Mean: {labels_mean}, Std: {labels_std}')

Mean: 12.902271270751953, Std: 6.502640724182129


In [22]:
labels_generator = (labels - labels_mean) / labels_std

### Label Discriminator Scale

In [23]:
labels_min = labels.min()
labels_max = labels.max()

In [24]:
labels_discriminator = labels - labels_mean
labels_discriminator = a + ((labels_discriminator - labels_min)*(b-a)) / (labels_max - labels_min)

# Save

In [25]:
np.save(f'../../data/preprocessed/{FILE_NAME}_chirps_scaled.npy', raw_data)
np.save(f'../../data/preprocessed/{FILE_NAME}_chirps_labels.npy', (labels_generator, labels_discriminator, labels))

# Plots

In [26]:
raw_data.mean()

-0.060282294

In [27]:
import seaborn as sns

#sns.boxplot((raw_data - raw_data.mean()).flatten())
#plt.figure(figsize=(10, 10))
#plt.show()